- https://www.youtube.com/watch?v=x1lAcT3xl5M&list=PLIUOU7oqGTLgQ7tCdDT0ARlRoh1127NSO&index=5
- https://github.com/jamescalam/transformers/blob/main/course/training/06_nsp_training.ipynb

In [176]:
import requests
from transformers import BertTokenizer, BertForPreTraining
import torch

In [177]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Link to the text
https://raw.githubusercontent.com/jamescalam/transformers/main/data/text/meditations/clean.txt

In [178]:
data = requests.get('https://raw.githubusercontent.com/jamescalam/transformers/main/data/text/meditations/clean.txt')

data
#response 200 is good

<Response [200]>

In [179]:
#we need to detach 
text = data.text.split('\n')

text[:3]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.']

Now we want to create a 50% - 50% of sentences that are random and not random
so we split every sentences by the period characters '.' and depend all of those to a new list, a flat list containing sentences, no paragraph, just sentences  

In [180]:
bag = [sentence for para in text for sentence in para.split('.') if sentence != '']
 
bag_size = len(bag) #so now we have 1372 sentences to sample from

Now we want to loop trough each paragraph within text, choose a sentence from each paragraph and 50% of the time select a sentence from the bag and append that to the end, so 50% of the time append the actual genuine net set onto it.
An then we create labels as to whether we have randomized it or not randomize it.

In [181]:
import random

# #inizialize
sentence_a = []
sentence_b = []
label = []

# #loop trough each paragraph in our text 
for paragraph in text:
    sentences = [
        sentence for sentence in paragraph.split('.') if sentence != ''
    ]
    #split each paragraph into sentences anche chek if these paragraph has more than 1 sentence 
    num_sentences = len(sentences)
    if num_sentences > 1: #we want to ignore the single sentence paragraph
        start = random.randint(0, num_sentences -2) #explanation below
        #50%-50% randomize or not randomize sentence b 
        if random.random() >= 0.5: #where random is between 0 and 1
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1]) #we just append the next sentence after a 
            label.append(0) #because this is the next sentence
        else:
            index = index = random.randint(0, bag_size-1)
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index]) #take a random sentence b from the bag
            label.append(1) #this is not the next sentence

Expalnation of:  #explanation below
        
Now we want to set the sentence that we will start from
in the case we use sentence a and b consecutively so we don't randomize sencence b, we want to make sure that we have enough space at the end of our sentences to take both sentence a and sentence b so so let's say we have a paragraph made by 
[0, 1, 2 ,3 ,4] 5 sentences in there 
we want to start from sentence a, if we select 4 then we don't have a sentence b to select from, so we sould say that the random number should be between 0 and the max -2 indices  


In [182]:
label[:3]

[0, 0, 0]

In [183]:
for i in range (3):
    print(label[i])
    print(sentence_a[i]+ '\n --')
    print(sentence_b[i]+ '\n')

0
 I observed that everybody believed that he thought as he spoke, and that in all that he did he never had any bad intention; and he never showed amazement and surprise, and was never in a hurry, and never put off doing a thing, nor was perplexed nor dejected, nor did he ever laugh to disguise his vexation, nor, on the other hand, was he ever passionate or suspicious
 --
 He was accustomed to do acts of beneficence, and was ready to forgive, and was free from all falsehood; and he presented the appearance of a man who could not be diverted from right rather than of a man who had been improved

0
 I observed too his habit of careful inquiry in all matters of deliberation, and his persistency, and that he never stopped his investigation through being satisfied with appearances which first present themselves; and that his disposition was to keep his friends, and not to be soon tired of them, nor yet to be extravagant in his affection; and to be satisfied on all occasions, and cheerful; a

So in the third case sentence b is not a continuation of sentence a, meanwhile in the first and secod case sentence b is a continuation of sentence a 
Now we can prepare our data and tokenize

In [184]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation = True, padding = 'max_length')
inputs.keys()

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [185]:
inputs
#sentence a has token_type_ids = 0 and sencence b has token_type_ids = 1

{'input_ids': tensor([[  101,  1045,  5159,  ...,     0,     0,     0],
        [  101,  1045,  5159,  ...,     0,     0,     0],
        [  101,  1045,  4067,  ...,  1998,  7280,   102],
        ...,
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0],
        [  101,  7887,  3288,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [186]:
#sentence a has token_type_ids = 0 and sencence b has token_type_ids = 1
inputs.token_type_ids[0]
#so first we have sentence a (the zeros)) follow by then sentence b (the ones) and then the remaining tokens are padding tokens

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [187]:
inputs.input_ids[0]
#where "102" is in the middle of our sentences a and b

tensor([  101,  1045,  5159,  2008,  7955,  3373,  2008,  2002,  2245,  2004,
         2002,  3764,  1010,  1998,  2008,  1999,  2035,  2008,  2002,  2106,
         2002,  2196,  2018,  2151,  2919,  6808,  1025,  1998,  2002,  2196,
         3662, 21606,  1998,  4474,  1010,  1998,  2001,  2196,  1999,  1037,
         9241,  1010,  1998,  2196,  2404,  2125,  2725,  1037,  2518,  1010,
         4496,  2001,  2566, 19386,  2098,  4496,  2139, 24455,  1010,  4496,
         2106,  2002,  2412,  4756,  2000, 14249,  2010,  2310, 18684,  3508,
         1010,  4496,  1010,  2006,  1996,  2060,  2192,  1010,  2001,  2002,
         2412, 13459,  2030, 10027,   102,  2002,  2001, 17730,  2000,  2079,
         4490,  1997,  3841, 12879,  6610,  5897,  1010,  1998,  2001,  3201,
         2000,  9641,  1010,  1998,  2001,  2489,  2013,  2035,  6270,  9021,
         1025,  1998,  2002,  3591,  1996,  3311,  1997,  1037,  2158,  2040,
         2071,  2025,  2022, 18356,  2013,  2157,  2738,  2084, 

so now we have input tensors but we also need label tensors

In [188]:
torch.LongTensor(label)
#this format is not the right one

tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
        0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
        0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
        1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,

In [189]:
inputs.input_ids
#we want a list whithin a list 

tensor([[  101,  1045,  5159,  ...,     0,     0,     0],
        [  101,  1045,  5159,  ...,     0,     0,     0],
        [  101,  1045,  4067,  ...,  1998,  7280,   102],
        ...,
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0],
        [  101,  7887,  3288,  ...,     0,     0,     0]])

In [190]:
inputs['labels'] = torch.LongTensor([label]).T

In [191]:
inputs.labels[:10]

tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1]])

so now for example the vector 
tensor([[  101,  2013, 21692,  ...,     0,     0,     0],
maches with the first vector there 
tensor([[0],

Now we have all the tensors we need to train our model and we need to set up the input pipeline for training. When we train we need to use a pythorch data loader object and to create that we need to create a pytorch dataset object from our data.

In [192]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [193]:
dataset = MeditationsDataset(inputs)

In [194]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [195]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [196]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-6)

In [197]:
from tqdm import tqdm  # for our progress bar

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/20 [00:00<?, ?it/s]/var/folders/8s/5jql9hyd6p7cp25xdxcgjp4c0000gn/T/ipykernel_15107/81523791.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  0%|          | 0/20 [00:10<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'backward'